# Project 80: Path sum: two ways

In a 5 by 5 matrix below, the minimal path sum from the top left to the bottom right, by only moving to the right and down, is the path with the lowest sum. 

Find the minimal path sum, in matrix.txt. As a 31K text file containing a 80 by 80 matrix, from the top left to the bottom right by only moving right and down.

# Solution

This is an almost text book problem for Djikstra's algorithm. We can intrepret the element of each matrix as a node connected by uni-directional paths going right and down. The distance of each path is simply the value of the element it lands on. 

A final consideration is that of the 00 element, i.e top-left. This is an additional value to add to the length of the path calculated to Djikstra's algorithm. 

In [136]:
from collections import defaultdict

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distance = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        assert from_node in self.nodes, "ERROR: starting value must be a node. "
        assert to_node in self.nodes, "ERROR: starting value must be a node. "

        self.edges[from_node].append(to_node)
        self.distance[(from_node, to_node)] = distance

In [324]:
import numpy as np
from collections import OrderedDict

def martix_to_graph(mat):
    """
        Turns a matrix into a graph with each element connect to the 
        one to the right and one below it. 
        
        A graph is stored as an order dictionary, such that each node is a dictionary
    """
    graph = Graph()
    # First set up all the first element 
    for y in xrange(mat.shape[0]):
        for x in xrange(mat.shape[1]):
            print "Adding node (y,x) = : ", (y,x), " = ", mat[(y,x)]
            graph.add_node((y,x))
        
    # Now create the connections 
    for y in xrange(mat.shape[0]):
        for x in xrange(mat.shape[1]):
            # Downward and rightward only, we can 
            # never go back upon ourselves
            if x < mat.shape[1]-1:
                graph.add_edge( (y,x),(y,x+1), mat[y,x+1])
                print "Adding deg: ", (y,x), " to ",(x+1,y), " = ", mat[y,x+1] 
                print "            ", mat[(y,x)], " to ",mat[(y,x+1)], " = ", mat[y,x+1] 
            if y < mat.shape[0]-1:
                graph.add_edge( (y,x),(y+1,x), mat[y+1,x])
                print "Adding deg: ", (y,x), " to ",(y+1,x), " = ",mat[y+1,x] 
                print "            ", mat[(y,x)], " to ",mat[(y+1,x)], " = ", mat[y+1,x] 
            
    return graph

Load the graph from the file

In [323]:
test = np.array([[131,673,234,103,18],
                 [201,96,342,965,150],
                 [630,803,746,422,111],
                 [537,699,497,121,956],
                 [805,732,524,37,331]
                 ])
# mat   = np.loadtxt("matrix.txt", delimiter=',')
graph = martix_to_graph(test)

Adding node (y,x) = :  (0, 0)  =  131
Adding node (y,x) = :  (0, 1)  =  673
Adding node (y,x) = :  (0, 2)  =  234
Adding node (y,x) = :  (0, 3)  =  103
Adding node (y,x) = :  (0, 4)  =  18
Adding node (y,x) = :  (1, 0)  =  201
Adding node (y,x) = :  (1, 1)  =  96
Adding node (y,x) = :  (1, 2)  =  342
Adding node (y,x) = :  (1, 3)  =  965
Adding node (y,x) = :  (1, 4)  =  150
Adding node (y,x) = :  (2, 0)  =  630
Adding node (y,x) = :  (2, 1)  =  803
Adding node (y,x) = :  (2, 2)  =  746
Adding node (y,x) = :  (2, 3)  =  422
Adding node (y,x) = :  (2, 4)  =  111
Adding node (y,x) = :  (3, 0)  =  537
Adding node (y,x) = :  (3, 1)  =  699
Adding node (y,x) = :  (3, 2)  =  497
Adding node (y,x) = :  (3, 3)  =  121
Adding node (y,x) = :  (3, 4)  =  956
Adding node (y,x) = :  (4, 0)  =  805
Adding node (y,x) = :  (4, 1)  =  732
Adding node (y,x) = :  (4, 2)  =  524
Adding node (y,x) = :  (4, 3)  =  37
Adding node (y,x) = :  (4, 4)  =  331
Adding deg:  (0, 0)  to  (1, 0)  =  673
             

IndexError: index 5 is out of bounds for axis 1 with size 5

## Dijkstra's Algorithm

Let the node at which we are starting be called the initial node. Let the distance of node Y be the distance from the initial node to Y. Dijkstra's algorithm will assign some initial distance values and will try to improve them step by step.

* Assign to every node a tentative distance value: set it to zero for our initial node and to infinity for all other nodes.


* Set the initial node as current. Mark all other nodes unvisited. Create a set of all the unvisited nodes called the unvisited set.

* For the current node, consider all of its neighbors and calculate their tentative distances. Compare the newly calculated tentative distance to the current assigned value and assign the smaller one. For example, if the current node A is marked with a distance of 6, and the edge connecting it with a neighbor B has length 2, then the distance to B (through A) will be 6 + 2 = 8. If B was previously marked with a distance greater than 8 then change it to 8. Otherwise, keep the current value.

* When we are done considering all of the neighbors of the current node, mark the current node as visited and remove it from the unvisited set. A visited node will never be checked again.
If the destination node has been marked visited (when planning a route between two specific nodes) or if the smallest tentative distance among the nodes in the unvisited set is infinity (when planning a complete traversal; occurs when there is no connection between the initial node and remaining unvisited nodes), then stop. The algorithm has finished.

* Otherwise, select the unvisited node that is marked with the smallest tentative distance, set it as the new "current node", and go back to step 3.


In [313]:
from copy import copy

def find_minimum_path(graph,start_node,final_node):
    """"""
    visited = [] # The set of all nodes that have been visited 
    unvisted_nodes = {}
    for node in graph.nodes:
        unvisted_nodes[node] = 1e10
    unvisted_nodes[start_node] = 0
    
    current_node = start_node 
    paths = defaultdict(list)
    path_histories = []
    paths[current_node] = [current_node ]
    
    while unvisted_nodes:    
        # Add the current node to the path 
        if paths[current_node][-1] != current_node:
            paths[current_node].append(current_node)
    
        # Store what every tentative path looks like. 
        path_histories.append(paths)
    
        # Evaluate how far we have travelled 
        current_distance = unvisted_nodes[current_node] 
        
        # If that node is the finial one then let's leave
        if current_node == final_node:
            break
        
        # Consider all neighbours connect - recall: 
        # edges is a dictionary of nodes and distances 
        for neighbour in graph.edges[current_node]:
            # We never revist visited nodes
            if neighbour in visited:
                continue 
                
            # Calculate the new distance this path would have  
            tentative_distance = current_distance + graph.distance[(current_node,neighbour)]
            if tentative_distance < unvisted_nodes[neighbour]:
                paths[neighbour] = copy(paths[current_node])
                paths[neighbour].append(neighbour)
                print  paths[neighbour]
                unvisted_nodes[neighbour] = tentative_distance

        # Now that we're done marking all neighbours
        # remove the current node and progress along 
        del unvisted_nodes[current_node] 
        # Mark it as visited
        visited.append(current_node)
        # Move to the node with the smallest distance
        current_node = min(unvisted_nodes, key=unvisted_nodes.get) 
        if current_node == final_node:
            break

    # Don't forget to update the total distsance with the distance to the final node        
    return paths[final_node],current_distance,paths,visited,path_histories

In [317]:
path,total_dist,paths,visited,path_histories =  find_minimum_path(graph, (0,0),(4,4))
print path
print total_dist

sum = 0
for node in path:
    sum += mat[node]
    print mat[node]
print sum

[(0, 0), (0, 1)]
[(0, 0), (1, 0)]
[(0, 0), (1, 0), (1, 1)]
[(0, 0), (1, 0), (2, 0)]
[(0, 0), (1, 0), (1, 1), (1, 2)]
[(0, 0), (1, 0), (1, 1), (2, 1)]
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)]
[(0, 0), (0, 1), (0, 2)]
[(0, 0), (1, 0), (2, 0), (3, 0)]
[(0, 0), (0, 1), (0, 2), (0, 3)]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
[(0, 0), (1, 0), (1, 1), (2, 1), (3, 1)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (3, 2)]
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)]
[(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (4, 1)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (2, 4)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (3, 3)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (3, 2), (4, 2)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (3, 3), (3, 4)]
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (3, 3), (4, 3)]
[]
2884
0


In [318]:
print graph.distance#[((3,4),(4,4))]

{((2, 1), (3, 1)): 699, ((0, 2), (1, 2)): 342, ((1, 0), (2, 0)): 630, ((1, 1), (2, 1)): 803, ((2, 0), (2, 1)): 803, ((3, 3), (3, 4)): 956, ((1, 2), (2, 2)): 746, ((2, 3), (2, 4)): 111, ((2, 1), (2, 2)): 746, ((3, 0), (3, 1)): 699, ((0, 3), (1, 3)): 965, ((2, 2), (2, 3)): 422, ((2, 3), (3, 3)): 121, ((1, 2), (1, 3)): 965, ((0, 1), (1, 1)): 96, ((3, 1), (3, 2)): 497, ((1, 0), (1, 1)): 96, ((3, 1), (4, 1)): 732, ((1, 1), (1, 2)): 342, ((3, 3), (4, 3)): 37, ((2, 2), (3, 2)): 497, ((0, 0), (1, 0)): 201, ((3, 2), (3, 3)): 121, ((1, 3), (1, 4)): 150, ((2, 0), (3, 0)): 537, ((0, 0), (0, 1)): 673, ((1, 3), (2, 3)): 422, ((3, 0), (4, 0)): 805, ((0, 3), (0, 4)): 18, ((3, 2), (4, 2)): 524, ((0, 1), (0, 2)): 234, ((0, 2), (0, 3)): 103}


### 